In [ ]:
# !git clone https://github.com/salesforce/LAVIS.git batch-inf/LAVIS/

In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
sagemaker_default_bucket = sess.default_bucket()
region = sess.boto_session.region_name

!cp batchinf.py batch-inf/
!cp blip2_coco.yaml batch-inf/
!cp envsets.sh batch-inf/
!cp s5cmd batch-inf/

In [ ]:
import time
from datetime import datetime
from sagemaker.estimator import Estimator

image_uri = f'633205212955.dkr.ecr.{region}.amazonaws.com/sagemaker-torch113-cu117-jre:latest'

instance_count = 1
instance_type = 'ml.g5.4xlarge' ## p4d - 8*40G / p4de - 8*80G

environment = {
    'ITERS': str(10),
    'BATCH_SIZE': str(200)
}

estimator = Estimator(role=role,
                      entry_point='envsets.sh',
                      source_dir='./batch-inf',
                      base_job_name='blip2-coco-inf',
                      instance_count=instance_count,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      max_run=5*3600, #任务最大存续时间，默认2day，需要提交ticket提升quota最大28天
                     )

estimator.fit()